# Proyecto  Final -  Parte 2

### **Parte 2.  Spark SQL**

- 1. [Preparación de la plataforma](#1)
- 2. [Carga,preparación y análisis mensual de los datos](#2)
    - 2.1 [Datos agua](#2.1)
    - 2.2 [Datos electricidad](#2.2)
    - 2.3 [Datos ficticios ](#2.3)
- 3. [Unión de los datasets y análisis](#3)  
    - 3.1 [Consumo agua en función de los Edificios](#3.1)
    - 3.2 [Consumo elecricidad en función de los Edificios](#3.2)

En esta segunda parte se va a llevar a cabo un análisis de los consumos mensuales tanto de agua como de electricidad en función de los meses y por otro lado su relación con respecto a aforos máximos

<a id="1"></a>
### * 1. Preparación de la plataforma. *


In [1]:
System.setSecurityManager(null)

In [2]:
import org.apache.spark.sql.SparkSession

In [3]:
val spark = SparkSession.builder.enableHiveSupport().getOrCreate()

spark = org.apache.spark.sql.SparkSession@1b004aa2


Spark local-1580501064108: Some(http://master:4041)

In [4]:
import spark.implicits._

In [5]:
import org.apache.spark.sql.types._

In [6]:
import org.apache.spark.sql.functions._

<a id="2"></a>
### * 2. Carga,preparación y análisis de los datos. *


<a id="2.1"></a>
### * 2.1 Datos agua. *

In [7]:
val d_agua= spark.read.format("csv").
option("delimiter", ";").
option("header", false). // los datos no incorporan cabecera
option("inferSchema", true).
load("hdfs:///eoi/Proyecto-Final/2013agua.csv").cache

d_agua = [_c0: string, _c1: double ... 5 more fields]


[_c0: string, _c1: double ... 5 more fields]

In [8]:
d_agua.printSchema

root
 |-- _c0: string (nullable = true)
 |-- _c1: double (nullable = true)
 |-- _c2: integer (nullable = true)
 |-- _c3: integer (nullable = true)
 |-- _c4: integer (nullable = true)
 |-- _c5: integer (nullable = true)
 |-- _c6: integer (nullable = true)



In [9]:
val d_agua_cabe = d_agua.withColumnRenamed("_c0", "Edificio").withColumnRenamed("_c1", "Consumo").withColumnRenamed("_c2", "Año").withColumnRenamed("_c3", "Mes").withColumnRenamed("_c4", "Día").withColumnRenamed("_c5", "Hora").withColumnRenamed("_c6", "Minuto")

d_agua_cabe = [Edificio: string, Consumo: double ... 5 more fields]


[Edificio: string, Consumo: double ... 5 more fields]

In [10]:
d_agua_cabe.count()

1004895

Como ya se analizó anteriormente en Spark Core la diferencia entre edificios y UPO en esta parte se decide no volver a analizar esa comparativa y analizar unicamente los edificios. Además se asume que los valores negativos en consumo con erróneos y por ello se eliminan.

In [11]:
val d_agua_EDIF_sin_neg = d_agua_cabe.filter($"Consumo" >= 0.0 && "$Edificio" != "AGUA_UPO_")

d_agua_EDIF_sin_neg = [Edificio: string, Consumo: double ... 5 more fields]


[Edificio: string, Consumo: double ... 5 more fields]

In [12]:
d_agua_EDIF_sin_neg.count()

1004748

In [13]:
d_agua_EDIF_sin_neg.show(5)

+-----------+-------+----+---+---+----+------+
|   Edificio|Consumo| Año|Mes|Día|Hora|Minuto|
+-----------+-------+----+---+---+----+------+
|AGUA_EDIF_1|    0.0|2013|  1|  1|   0|     0|
|AGUA_EDIF_1|    0.0|2013|  1|  1|   0|    15|
|AGUA_EDIF_1|    0.0|2013|  1|  1|   0|    30|
|AGUA_EDIF_1|    0.0|2013|  1|  1|   0|    45|
|AGUA_EDIF_1|    0.0|2013|  1|  1|   1|     0|
+-----------+-------+----+---+---+----+------+
only showing top 5 rows



### *Cálculo del consumo de agua mensual *

Dado que en Spark Core se han calculado los consumos por edificios, en este caso se calculan los consumos en función de los meses

In [14]:
val agua_mes = d_agua_EDIF_sin_neg.groupBy($"Mes")
    .agg(sum($"Consumo").as("Consumo mensual"))
    .sort($"Consumo mensual".desc)

agua_mes = [Mes: int, Consumo mensual: double]


[Mes: int, Consumo mensual: double]

In [15]:
agua_mes.show()

+---+------------------+
|Mes|   Consumo mensual|
+---+------------------+
| 10| 6553.194999999841|
| 11| 6348.106999999831|
|  6| 6243.957999999862|
|  5| 6085.569999999883|
|  9| 6005.066999999882|
|  4| 5424.244999999892|
| 12|5223.0379999999295|
|  7|5222.9399999999005|
|  1| 5100.248999999954|
|  2|  4779.97699999989|
|  3| 4550.615999999952|
|  8|3631.9559999999583|
+---+------------------+



Se observa como los meses en los que más agua se consumo son octubre y noviembre y el mes que menos es Agosto, el último tiene una explicación bastante lógica y es que en el mes de Agosto la mayoría de la gente se encuentra de vacaciones

<a id="2.2"></a>
### * 2.2. Datos electricidad. *

In [16]:
val d_elec= spark.read.format("csv").
option("delimiter", ";").
option("header", false). // los datos no incorporan cabecera
option("inferSchema", true).
load("hdfs:///eoi/Proyecto-Final/2013electricidad.csv").cache

d_elec = [_c0: string, _c1: double ... 5 more fields]


[_c0: string, _c1: double ... 5 more fields]

In [17]:
d_elec.printSchema

root
 |-- _c0: string (nullable = true)
 |-- _c1: double (nullable = true)
 |-- _c2: integer (nullable = true)
 |-- _c3: integer (nullable = true)
 |-- _c4: integer (nullable = true)
 |-- _c5: integer (nullable = true)
 |-- _c6: integer (nullable = true)



In [18]:
val d_elec_cabe = d_elec.withColumnRenamed("_c0", "Edificio").withColumnRenamed("_c1", "Consumo").withColumnRenamed("_c2", "Año").withColumnRenamed("_c3", "Mes").withColumnRenamed("_c4", "Día").withColumnRenamed("_c5", "Hora").withColumnRenamed("_c6", "Minuto")

d_elec_cabe = [Edificio: string, Consumo: double ... 5 more fields]


[Edificio: string, Consumo: double ... 5 more fields]

In [19]:
d_elec_cabe.count()

998289

Como ya se analizó anteriormente en Spark Core la diferencia entre edificios y UPO en esta parte se decide no volver a analizar esa comparativa y analizar unicamente los edificios. Además se asume que los valores negativos en consumo con erróneos y por ello se eliminan.

In [20]:
val d_ELEC_EDIF_sin_neg = d_elec_cabe.filter($"Consumo" >= 0.0 && "$Edificio" != "ELEC_UPO_")

d_ELEC_EDIF_sin_neg = [Edificio: string, Consumo: double ... 5 more fields]


[Edificio: string, Consumo: double ... 5 more fields]

In [21]:
d_ELEC_EDIF_sin_neg.count()

996068

In [22]:
d_ELEC_EDIF_sin_neg.show(5)

+-----------+-------+----+---+---+----+------+
|   Edificio|Consumo| Año|Mes|Día|Hora|Minuto|
+-----------+-------+----+---+---+----+------+
|ELEC_EDIF_1|  1.354|2013|  1|  1|   0|     0|
|ELEC_EDIF_1|  1.521|2013|  1|  1|   0|    15|
|ELEC_EDIF_1|  1.239|2013|  1|  1|   0|    30|
|ELEC_EDIF_1|  1.347|2013|  1|  1|   0|    45|
|ELEC_EDIF_1|    1.3|2013|  1|  1|   1|     0|
+-----------+-------+----+---+---+----+------+
only showing top 5 rows



### *Cálculo del consumo de electricidad mensual *

In [23]:
val elec_mes = d_ELEC_EDIF_sin_neg.groupBy($"Mes")
    .agg(sum($"Consumo").as("Consumo mensual"))
    .sort($"Consumo mensual".desc)

elec_mes = [Mes: int, Consumo mensual: double]


[Mes: int, Consumo mensual: double]

In [24]:
elec_mes.show()

+---+------------------+
|Mes|   Consumo mensual|
+---+------------------+
|  9|2052214.4620000173|
| 10|1880115.5610000044|
| 12|1841825.4030000065|
| 11|1764695.5759999983|
|  7|1712505.8759999909|
|  1|1529654.0739999951|
|  6| 1512178.091000014|
|  2|1422766.8090000013|
|  3|1300775.5259999982|
|  5|1289801.8820000004|
|  4|1099519.8939999992|
|  8| 935923.2849999974|
+---+------------------+



Se observa como los meses en los que más electricidad se consume son Septiembre, Octubre y Diciembre, esto es debido a que en estos meses existe menos luz natural y es necesario consumir una mayor cantidad de luz artificial

<a id="2.3"></a>
### * 2.2. Datos ficticios. *

In [25]:
val d_ficticios= spark.read.format("csv").
option("delimiter", ";").
option("header", true). 
option("inferSchema", true).
load("hdfs:///eoi/Proyecto-Final/Datos_ficticios.csv").cache

d_ficticios = [Edificio: string, Direccion: string ... 2 more fields]


[Edificio: string, Direccion: string ... 2 more fields]

In [26]:
d_ficticios.count()

29

In [27]:
d_ficticios.show(29)

+---------------+---------------+---------+-----------+
|       Edificio|      Direccion|Provincia|AforoMaximo|
+---------------+---------------+---------+-----------+
|         EDIF_1|Direccion upo-1|  Sevilla|      18802|
|EDIF_1_CAFETERI|Direccion upo-1|  Sevilla|       3571|
|        EDIF_10|Direccion upo-1|  Sevilla|      15912|
|        EDIF_11|Direccion upo-1|  Sevilla|      15176|
|        EDIF_12|Direccion upo-1|  Sevilla|      18747|
|        EDIF_13|Direccion upo-1|  Sevilla|      14833|
|        EDIF_14|Direccion upo-1|  Sevilla|      16843|
|        EDIF_15|Direccion upo-1|  Sevilla|       6698|
|        EDIF_16|Direccion upo-1|  Sevilla|      19598|
|        EDIF_17|Direccion upo-1|  Sevilla|       6314|
|EDIF_17_COMEDOR|Direccion upo-1|  Sevilla|      12827|
| EDIF_18_KIOSCO|Direccion upo-1|  Sevilla|      12557|
|         EDIF_2|Direccion upo-2|  Sevilla|       8878|
|        EDIF_20|Direccion upo-2|  Sevilla|       8375|
|        EDIF_21|Direccion upo-2|  Sevilla|     

In [28]:
d_ficticios.printSchema

root
 |-- Edificio: string (nullable = true)
 |-- Direccion: string (nullable = true)
 |-- Provincia: string (nullable = true)
 |-- AforoMaximo: integer (nullable = true)



<a id="3"></a>
### * 3. Unión de los datasets y análisis. *

<a id="3.1"></a>
### * 3.1 Consumo agua en función de los edificios. *

Se decide modificar un poco la estructura de los datasets y crear una columna de tipo Time Stamp que agrupe la fecha en una única variable, con el fin de que al análizar la unión sea más visual y además cambiar el nombre de Edificio ya que si no coincidía con el datos ficticios y no se podría llevar a cabo la union

Se crean los encabezamos del dataframe

In [29]:
val db_agua_5 = d_agua.toDF("Edificio agua", "Consumo m3", "Año", "Mes", "Día", "Hora","Minutos")

db_agua_5 = [Edificio agua: string, Consumo m3: double ... 5 more fields]


[Edificio agua: string, Consumo m3: double ... 5 more fields]

In [30]:
val db_agua_4 = db_agua_5.select($"Edificio agua",$"Consumo m3".cast("Double"),(concat($"Año", lit("-"), 
                                          $"Mes", lit("-"), 
                                          $"Día", lit(" "), 
                                          $"Hora", lit(":"),
                                          $"MinutoS").cast("Timestamp").as("Fecha")))

db_agua_4 = [Edificio agua: string, Consumo m3: double ... 1 more field]


[Edificio agua: string, Consumo m3: double ... 1 more field]

Para poder unirlos se realiza el mismo paso que en Core quitandole AGUA_

In [31]:
val db_agua_3 = db_agua_4.withColumn("Edificio agua", regexp_replace(db_agua_4("Edificio agua"),"AGUA_", "" ))

db_agua_3 = [Edificio agua: string, Consumo m3: double ... 1 more field]


[Edificio agua: string, Consumo m3: double ... 1 more field]

In [32]:
val db_agua_2 = db_agua_3.filter($"Consumo m3" >= 0.0 && "$Edificio agua" != "AGUA_UPO")

db_agua_2 = [Edificio agua: string, Consumo m3: double ... 1 more field]


[Edificio agua: string, Consumo m3: double ... 1 more field]

In [33]:
val db_agua_edificio = db_agua_2.groupBy($"Edificio agua")
    .agg(sum($"Consumo m3").as("Consumo mensual m3"))
    .sort($"Consumo mensual m3".desc)

db_agua_edificio = [Edificio agua: string, Consumo mensual m3: double]


[Edificio agua: string, Consumo mensual m3: double]

<a id="3.2"></a>
### * 3.1 Consumo electricidad en función de los edificios. *

In [34]:
val db_elec_4 = d_elec.toDF("Edificio elec", "Consumo kW/h", "Año", "Mes", "Día", "Hora","Minutos")

db_elec_4 = [Edificio elec: string, Consumo kW/h: double ... 5 more fields]


[Edificio elec: string, Consumo kW/h: double ... 5 more fields]

In [35]:
val db_elec_3 = db_elec_4.select($"Edificio elec",$"Consumo kW/h".cast("Double"),(concat($"Año", lit("-"), 
                                          $"Mes", lit("-"), 
                                          $"Día", lit(" "), 
                                          $"Hora", lit(":"),
                                          $"MinutoS").cast("Timestamp").as("Fecha")))

db_elec_3 = [Edificio elec: string, Consumo kW/h: double ... 1 more field]


[Edificio elec: string, Consumo kW/h: double ... 1 more field]

In [36]:
val db_elec_2 = db_elec_3.filter($"Consumo kW/h" >= 0.0 && "$Edificio elec" != "ELEC_UPO")

db_elec_2 = [Edificio elec: string, Consumo kW/h: double ... 1 more field]


[Edificio elec: string, Consumo kW/h: double ... 1 more field]

In [37]:
val db_elec = db_elec_2.withColumn("Edificio elec", regexp_replace(db_elec_2("Edificio elec"),"ELEC_", "" ))

db_elec = [Edificio elec: string, Consumo kW/h: double ... 1 more field]


[Edificio elec: string, Consumo kW/h: double ... 1 more field]

In [38]:
val db_elec_edificio = db_elec.groupBy($"Edificio elec")
    .agg(sum($"Consumo kW/h").as("Consumo mensual kW/h"))
    .sort($"Consumo mensual kW/h".desc)

db_elec_edificio = [Edificio elec: string, Consumo mensual kW/h: double]


[Edificio elec: string, Consumo mensual kW/h: double]

### Unión

In [39]:
val join_1 = db_elec_edificio.join(db_agua_edificio, db_elec_edificio.col("Edificio elec") === db_agua_edificio.col("Edificio agua"), "inner")

join_1 = [Edificio elec: string, Consumo mensual kW/h: double ... 2 more fields]


[Edificio elec: string, Consumo mensual kW/h: double ... 2 more fields]

In [40]:
join_1.show(29)

+---------------+--------------------+---------------+------------------+
|  Edificio elec|Consumo mensual kW/h|  Edificio agua|Consumo mensual m3|
+---------------+--------------------+---------------+------------------+
|         EDIF_3|  173988.71000000104|         EDIF_3| 952.7199999998963|
|         EDIF_8|  53756.026000000435|         EDIF_8| 348.9499999999778|
|        EDIF_42|   6325.060999998495|        EDIF_42| 6.159999999999969|
|        EDIF_16|  23105.522000000114|        EDIF_16| 4197.345999999584|
|EDIF_1_CAFETERI|  123496.35400000021|EDIF_1_CAFETERI| 709.8699999999565|
|        EDIF_10|  224691.37899999702|        EDIF_10| 628.2499999999555|
|        EDIF_12|   82013.96899999968|        EDIF_12| 396.7199999999699|
|        EDIF_13|  27464.337000000156|        EDIF_13|126.06000000000698|
|         EDIF_6|   135662.6199999995|         EDIF_6| 479.7899999999386|
|        EDIF_14|  252734.93899999987|        EDIF_14| 301.3999999999884|
|        EDIF_21|   579830.6299999943|

In [41]:
val join_2 = join_1.join(d_ficticios, join_1.col("Edificio agua") === d_ficticios.col("Edificio"), "inner")

join_2 = [Edificio elec: string, Consumo mensual kW/h: double ... 6 more fields]


[Edificio elec: string, Consumo mensual kW/h: double ... 6 more fields]

In [42]:
val total_1 = join_2.select( "Edificio", "Consumo mensual kW/h", "Consumo mensual m3", "AforoMaximo")

total_1 = [Edificio: string, Consumo mensual kW/h: double ... 2 more fields]


[Edificio: string, Consumo mensual kW/h: double ... 2 more fields]

In [43]:
val total_final = total_1.sort($"AforoMaximo")

total_final = [Edificio: string, Consumo mensual kW/h: double ... 2 more fields]


[Edificio: string, Consumo mensual kW/h: double ... 2 more fields]

In [44]:
total_final.show(29)

+---------------+--------------------+------------------+-----------+
|       Edificio|Consumo mensual kW/h|Consumo mensual m3|AforoMaximo|
+---------------+--------------------+------------------+-----------+
|            UPO|1.2911443386999983E7|39388.879999999896|       null|
|        EDIF_42|   6325.060999998495| 6.159999999999969|       3154|
|EDIF_1_CAFETERI|  123496.35400000021| 709.8699999999565|       3571|
|        EDIF_17|  219905.69300000125| 1219.489999999943|       6314|
|        EDIF_15|   263737.3690000025|3045.9600000001237|       6698|
|         EDIF_6|   135662.6199999995| 479.7899999999386|       6706|
|        EDIF_43|   7829.521000000247| 337.2700000000003|       7422|
|        EDIF_20|  1625734.7639999923| 6367.870000000127|       8375|
|         EDIF_2|    218905.908000002| 369.8799999999802|       8878|
|        EDIF_31|   57470.26199999967|  90.3800000000023|      10095|
|        EDIF_45|  285486.19299999875|               0.0|      10785|
|        EDIF_44|  1

En esta tabla se pueden analizar los consumos totales en función de los edificios y ver que aforo máximo poseen los mismos.

Se aprecia como no existe una relación directa entre consumos y aforo máximo. El edificio 45 es llamativo de nuevo, ya que no posee consumo de agua pero es uno de los que posee mayor consumo de electricidad y se podría afirmar que su aforo es medio. 

## Ejemplo de guardado de datos 

Se realiza un unico guardado  a modo de ejemplo, ya que el objetivo de este proyecto no es guardar todas las tablas

In [49]:
spark.sql("create database ProyectoFinal")

Name: org.apache.spark.sql.AnalysisException
Message: org.apache.hadoop.hive.metastore.api.AlreadyExistsException: Database proyectofinal already exists;
StackTrace:   at org.apache.spark.sql.hive.HiveExternalCatalog.withClient(HiveExternalCatalog.scala:106)
  at org.apache.spark.sql.hive.HiveExternalCatalog.doCreateDatabase(HiveExternalCatalog.scala:163)
  at org.apache.spark.sql.catalyst.catalog.ExternalCatalog.createDatabase(ExternalCatalog.scala:69)
  at org.apache.spark.sql.catalyst.catalog.SessionCatalog.createDatabase(SessionCatalog.scala:207)
  at org.apache.spark.sql.execution.command.CreateDatabaseCommand.run(ddl.scala:70)
  at org.apache.spark.sql.execution.command.ExecutedCommandExec.sideEffectResult$lzycompute(commands.scala:70)
  at org.apache.spark.sql.execution.command.ExecutedCommandExec.sideEffectResult(commands.scala:68)
  at org.apache.spark.sql.execution.command.ExecutedCommandExec.executeCollect(commands.scala:79)
  at org.apache.spark.sql.Dataset$$anonfun$6.apply

In [50]:
d_ficticios.write.saveAsTable("ProyectoFinal.datos")

lastException = null


Name: org.apache.spark.sql.AnalysisException
Message: Table `ProyectoFinal`.`datos` already exists.;
StackTrace:   at org.apache.spark.sql.DataFrameWriter.saveAsTable(DataFrameWriter.scala:393)

In [51]:
spark.catalog.listTables().show()

+-----+-------------+-----------+---------+-----------+
| name|     database|description|tableType|isTemporary|
+-----+-------------+-----------+---------+-----------+
|datos|proyectofinal|       null|  MANAGED|      false|
+-----+-------------+-----------+---------+-----------+



lastException: Throwable = null


In [48]:
spark.catalog.setCurrentDatabase("ProyectoFinal")